In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
% cd './drive/My Drive/'
% ls

/content/drive/My Drive
'Colab Notebooks'/   dermatology/   skin/


In [0]:
import os
import pandas as pd

from glob import glob
from sklearn.model_selection import train_test_split

def data_split(data_dir='/content/drive/My Drive/skin'):
  """ Splits images in data_dir into
  train, val, test
  80,20,20
  """
  filenames = glob(os.path.join(data_dir, '*','*.jpg'))
  df = pd.DataFrame(data=filenames, columns=['filenames'])
  df['label'] = df['filenames'].str.extract(r'skin\W(\w+)\W')
  df['label_code'] = pd.Categorical(df['label']).codes

  train_val, test = train_test_split(df, test_size=0.2, random_state=42)
  train_val, test = train_val.reset_index(drop=True), test.reset_index(drop=True)
  train, val = train_test_split(train_val, test_size=0.2, random_state=42)
  train, val = train.reset_index(drop=True), val.reset_index(drop=True)

  return train, val, test

In [0]:
#def data_split(
    """ Splits images in data_dir into
train, val, test
80,20,20
"""
train_val, test = train_test_split(df, test_size=0.2, random_state=42)
train_val, test = train_val.reset_index(drop=True), test.reset_index(drop=True)
train, val = train_test_split(train_val, test_size=0.2, random_state=42)
train, val = train.reset_index(drop=True), val.reset_index(drop=True)

#return train, val, test

In [0]:
import os
import pandas as pd
from glob import glob

def get_unknown_df(data_dir='/content/drive/My Drive/dermatology'):
  filenames = glob(os.path.join(data_dir, '*', '*','*.jpg'))
  df = pd.DataFrame(data=filenames, columns=['filenames'])
  df['label_code'] = 3 #Label can NOT be NONE

  return df

In [5]:
! pip install torch
! pip install efficientnet-pytorch

import torch

from efficientnet_pytorch import EfficientNet

def mymodel(unfreeze=True, num_classes=2):
  """Unfreeze(True) all the model weights or freeze(False) the convolutional 
  layers only
  """
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)
  model = model.to(device)
  for param in model.parameters():
    param.requires_grad = unfreeze
  
  return model

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=c2caef6faf2a6c4f5ffb6815aed5d6e804f81dc030654917e5696b973bcb7d15
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [0]:
def get_num_parameters(model):
  """Get the total number of parameters in a neural network
  """
  total_params = sum(p.numel() for p in model.parameters())
  total_trainable_params = sum(
  p.numel() for p in model.parameters() if p.requires_grad)
  
  return total_params, total_trainable_params

In [0]:
import torchvision
from torch.utils.data import DataLoader,Dataset
from PIL import Image

# Custom Pytorch dataloader for this dataset
class Derm(Dataset):
    """Reads a pandas dataframe with the images paths
    and labels"""
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load image data and get label
        X = Image.open(self.df['filenames'][index]).convert('RGB')
        y = torch.tensor(int(self.df['label_code'][index]))
        
        if self.transform:
            X = self.transform(X)

        return X, y

In [0]:
from torchvision import transforms

def get_loaders(size=224, batch_size=1,num_workers=1):
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Transforms
    data_transforms = {x : transforms.Compose([transforms.Resize(size),
                                              transforms.CenterCrop((size,size)),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean,std)]) for x in ['train', 'val', 'test', 'unknown']}

    # Dataframes
    df = {'train': train, 'val': val, 'test': test, 'unknown': unknown}                                    

    # Datasets
    image_datasets = {x: Derm(df[x], transform=data_transforms[x]) for x in ['train', 'val', 'test', 'unknown']}

    # Dataloaders
    dataloaders = {x: DataLoader(image_datasets[x], batch_size, num_workers) for x in ['train', 'val', 'test', 'unknown']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test', 'unknown']}
    
    return data_transforms, df, image_datasets, dataloaders, dataset_sizes

In [0]:
#data = Derm(df['train'],transform=transforms.Compose([transforms.Resize(size),
#                                           transforms.CenterCrop((size,size)),
#                                           transforms.ToTensor()
#                                           ,
#                                           transforms.Normalize(mean,std)
#                                           ]))
#
#X, y = next(iter(DataLoader(data,batch_size=4,shuffle=True,num_workers=4)))
#print(X.shape)
#print(X)

In [0]:
import time
import copy

def train_model(model, criterion, optimizer, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)

    return model

In [0]:
def predict_class(best_model, phase = 'test'):
  """Predict performance on the whole test set and predict class
  on the unknown set
  """
  if phase == 'test':
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloaders[phase]:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            best_model = best_model.to(device)
            outputs = best_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
  elif phase == 'unknown':
    with torch.no_grad():
        for data in dataloaders[phase]:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            best_model = best_model.to(device)
            outputs = best_model(images)
            _, predicted = torch.max(outputs.data, 1)
    return outputs, predicted    

In [12]:
# Main
import torch.optim as optim

from torch import nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = mymodel(unfreeze=True, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [13]:
# Main
train, val, test = data_split(data_dir='/content/drive/My Drive/skin')
unknown = get_unknown_df(data_dir='/content/drive/My Drive/dermatology')

total_params, total_trainable_params = get_num_parameters(model)
data_transforms, df, image_datasets, dataloaders, dataset_sizes = get_loaders(size=224, batch_size=4,num_workers=4)

num_epochs = 5
best_model = train_model(model, criterion, optimizer, num_epochs)
predict_class(best_model, phase = 'test')
predict_class(best_model, phase = 'unknown')

Epoch 0/4
----------
train Loss: 0.4310 Acc: 0.8112
val Loss: 0.2731 Acc: 0.9681

Epoch 1/4
----------
train Loss: 0.2347 Acc: 0.9388
val Loss: 0.1904 Acc: 0.9787

Epoch 2/4
----------
train Loss: 0.1916 Acc: 0.9335
val Loss: 0.1626 Acc: 0.9574

Epoch 3/4
----------
train Loss: 0.1462 Acc: 0.9495
val Loss: 0.1540 Acc: 0.9468

Epoch 4/4
----------
train Loss: 0.1594 Acc: 0.9468
val Loss: 0.1332 Acc: 0.9574

Training complete in 4m 17s
Best val Acc: 0.978723
Accuracy of the network on the test images: 97 %


(tensor([[ 0.3060, -0.2082],
         [ 1.4482, -1.0845],
         [ 0.6593, -1.0635]], device='cuda:0'),
 tensor([0, 0, 0], device='cuda:0'))

In [0]:
# To print to log
print('train', train.shape)
print('test', test.shape)
print('val', val.shape)
print('unknown', unknown.shape)
print(f'{total_params:,} total parameters.')
print(f'{total_trainable_params:,} training parameters.')
image, label = next(iter(dataloaders['train']))
print(image.shape)
print(label.shape)

train (183, 3)
test (58, 3)
val (46, 3)
unknown (1, 2)
4,010,110 total parameters.
4,010,110 training parameters.
torch.Size([4, 3, 224, 224])
torch.Size([4])
